In [1]:
import pandas as pd
import numpy as np
from scipy import sparse

In [2]:
ratings=pd.read_csv('BX-Book-Ratings.csv',sep=';', encoding='latin-1')
books = pd.read_csv('BX-Books_mod.csv',sep=';', encoding='latin-1')
users = pd.read_csv('BX-Users.csv', sep=';', encoding='latin-1')

In [3]:
n_user,n_item=(map(len, [ratings['User-ID'].unique(),ratings['ISBN'].unique()]))

In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
ratings['book_ID']=le.fit_transform(ratings.ISBN)
ratings['user_ID']=le.fit_transform(ratings['User-ID'])

In [5]:
ratings.columns

Index(['User-ID', 'ISBN', 'Book-Rating', 'book_ID', 'user_ID'], dtype='object')

In [6]:
piv=sparse.lil_matrix((n_user,n_item))

In [7]:
for row in ratings.itertuples():
    piv[row[5],row[4]]=row[3]+ 1e-9

In [14]:
ratings[ratings['user_ID']==nm]

,User-ID,ISBN,Book-Rating,book_ID,user_ID
587065,141794,0141301066,0,27718,53648
587066,141794,0373834462,0,71413,53648
587067,141794,0375725849,0,74204,53648
587068,141794,0385721250,0,83612,53648
587069,141794,0449213447,0,111304,53648
587070,141794,0486227944,8,121270,53648
587071,141794,0552527521,0,130209,53648
587072,141794,0553577905,0,139222,53648
587073,141794,0749316845,0,185744,53648
587074,141794,0812555198,0,209397,53648


In [8]:
def take_most_commun(piv,user,number):
    dic={}
    h = piv[user,:].nonzero()[1]
    for i in range(piv.shape[0]):
        z=0
        dist=0
        if i != user:
            o=piv[i,:].nonzero()[1]
            for col in np.intersect1d(h,o):
                z+=1
                dist+= abs(piv[user,col]-piv[i,col])
            if z!=0:
                dic[i]=[z,dist/z ,z**2/(dist+1)]
    if len(dic)==0:
        return None
    data=pd.DataFrame.from_dict(dic, orient='index')
    data.columns=['comune_columns','distance','my_index']
    if data.shape[0]>number:
        return data.sort_values(['comune_columns','distance'],axis=0, ascending=[0,1])[:number]
    else:
        return data.sort_values(['comune_columns','distance'],axis=0, ascending=[0,1])
    '''if len(dic)>number:
        return [(i,dic[i]) for i in sorted(dic, key=dic.__getitem__, reverse=1)[:number]]
    else:
        return [(i,dic[i]) for i in sorted(dic, key=dic.__getitem__, reverse=1)]'''

In [9]:
nm=53648

In [10]:
m=take_most_commun(piv,100,100)

In [11]:
m

,comune_columns,distance,my_index
32221,1,6.0,0.142857
87422,1,6.0,0.142857
35813,1,6.0,0.142857


In [20]:
pred_rating_mean_user_based(100,100,piv,100)

6.0

In [12]:
def pred_rating_mean_user_based(user,item,piv,neighbors):
    closer=take_most_commun(piv,user,neighbors)
    rat=0
    if str(closer)=='None':
        return round(piv[:,item].sum()/piv[:,item].count_nonzero(),3)
    else:
        for i in closer.index:
            z=0
            if piv[i,item]!=0:
                rat+=piv[i,item]
                z+=1
        if z==0:
            return round(piv[user,:].sum()/piv[user,:].count_nonzero(),3)
        else:
            return round(rat/z ,3)

In [ ]:
def pred_rating_mean_user_based2(user,item,piv,closer):
    rat=0
    if closer==None:
        return  round(piv[:,item].sum()/piv[:,item].count_nonzero(),3)
    else:
        for i in closer.index:
            z=0
            if piv[i,item]!=0:
                rat+=piv[i,item]
                z+=1
        if z==0:
            return round(piv[user,:].sum()/piv[user,:].count_nonzero(),3)
        else:
            return round(rat/z ,3)

In [15]:
pred_rating_mean_user_based(nm,331192,piv,1000)

1.7609999999999999

In [24]:
for i,l in ratings.user_ID.iteritems(): 
    if i <10:
        print(i,l)

0 104433
1 104434
2 104435
3 104436
4 104436
5 104437
6 104438
7 104439
8 104440
9 104441


In [ ]:
pred_rating_mean_user_based(100,100,piv,100)

In [ ]:
X,y=ratings[['User-ID','ISBN']],ratings['Book-Rating']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_tr , x_te, y_tr, y_te = train_test_split(X,y,test_size=0.2, random_state=0)

In [ ]:
def create_pivot_UB(X,y,Nu,Nb):
    #Create two user-item matrices
    #n_users = X['User_Lab'].unique().shape[0]
    #n_items = X['Book_Lab'].unique().shape[0]
    #print('us',n_users,'it',n_items)
    rat=pd.concat([X,y],axis=1)
    pivot =sparse.lil_matrix((Nu, Nb))
    for line in rat.itertuples():
        pivot[line[3], line[4]] = line[5]+1e-5
    return pivot

In [ ]:
piv= create_pivot_UB(x_tr,y_tr,n_user,n_item)

In [ ]:
x_tr['user'], x_tr['book']=(map(le.fit_transform, [x_tr['User-ID'],x_tr['ISBN']]))

In [ ]:
y_tr[1]

In [ ]:
x_tr.iloc[0,0]

In [ ]:
len(y_tr)

In [ ]:
x_tr.shape

In [16]:
clos==None

True

In [18]:
if clos==None:print('ook')

ook


In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [66]:
l=pd.Series(cos)
l.sort_values(ascending=0)[:10]

28609     1.0
332607    1.0
289648    1.0
333778    1.0
332632    1.0
20594     1.0
332627    1.0
332626    1.0
332624    1.0
332623    1.0
dtype: float64

In [62]:
cos.shape

(340556,)

In [56]:
piv.shape

(105283, 340556)

In [69]:
def mean(elem, pivot):
    res = pivot[:,elem].sum()/pivot[:,elem].count_nonzero()
    return res

In [61]:
cos=cosine_similarity(piv[:,1].T,piv.T).flatten()

In [67]:
def take_item_similarity(item,piv, n):
    cos=cosine_similarity(piv[:,item].T,piv.T).flatten()
    l=pd.Series(cos).sort_values(ascending=0)[:n]
    return l

In [71]:
def pred_item_based_cosine_deviation(user,item,piv,closer):
        m = mean(item, piv)
        somma=0
        for it in closer.index:
            somma+= closer[it]*(piv[user,it]-mean(it,piv))
        z=somma/closer.sum()
        if z!=z: z=0
        return round(float(m+z),3)

In [75]:
s= take_item_similarity(10,piv,10)

In [76]:
pred_item_based_cosine_deviation(100,10,piv,s)

-3.25

In [106]:
float(cosine_similarity(piv[:,3].T,piv[:,10].T))

1.428571428367347e-10

In [111]:
def find_similar_item_rated(user,item,piv,n=10):
    col = piv[user,:].nonzero()[1]
    sim={}
    for i in col:
        sim[i]= float(cosine_similarity(piv[:,i].T,piv[:,item].T))
    return [(i,sim[i]) for i in sorted(sim, key=sim.__getitem__, reverse=1)[:n]]

In [114]:
azz=find_similar_item_rated(100,100,piv)

In [90]:
s=sparse.lil_matrix((2,3))

In [81]:
piv[,:].nonzero()[1]

array([   231,  10495,  72058,  84148,  90587,  95565, 162072, 164429,
       181274, 193150, 193515, 204595, 232598, 250730, 254023, 259485,
       263545, 276735], dtype=int32)

In [124]:
def find_rating_ITBMean(user,similar,piv):
    so=0
    c=0
    for el in similar: 
        if round(el[1],2)!=0.00:
            c+=1
            so+= piv[user,el[0]]
    if c==0:
        return 0
    else:
        return round(so/c, 2)

In [125]:
find_rating_ITBMean(100,azz,piv)

0

In [123]:
np.mean([azz[l][1] for l in range(len(azz)) ])

0.0

In [95]:
s[1,:].nonzero()[1]

array([], dtype=int32)

In [1]:
def tolgo_media(piv):
    for i in range(piv.shape[0]):
        for el in piv[i,:].nonzero()[1]:
            piv[i,el] -= mean(i,piv)
    return piv

In [12]:
piv[1,:].nonzero()[1]

array([56363], dtype=int32)

In [30]:
piv[78,:].nonzero()

(array([0], dtype=int32), array([23524], dtype=int32))

In [31]:
piv[78,23524]

8.0000000010000001

In [21]:
import Lil_lib as lb1

In [27]:
lb1.mean(78,piv)

8.0000000010000001